In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.special import gammaln
from myfunc import *

In [66]:
# Loading ALL Function Definitions
def load_and_preprocess_data(file_path):
    """
    Loads the CSV data, preprocesses it by adding month, date, and birthday columns,
    and filters data for customers with a specific birthday.
    """
    df = pd.read_csv(file_path)
    df['month'] = pd.to_datetime(df['date']).dt.strftime('%b')
    df['date'] = pd.to_datetime(df['date'])
    df['birthday'] = pd.to_datetime(df['birthday'])
    df['spend_rand'] = np.random.uniform(20, 40, size=len(df))


    # Filter DataFrame for customers with birthday '1/1/2018'
    birthday_filter = pd.Timestamp('2018-01-01')
    df_filtered = df[df['birthday'] == birthday_filter]
    df_filtered.sort_values(by='date', inplace=True)
    return df_filtered

def calculate_distinct_customer_count(df):
    """
    Groups data by date and counts distinct customer IDs.
    """
    grouped_df = df.groupby('date').agg({
        'cust_id': pd.Series.nunique,  # Count distinct customer IDs
        'spend_rand': 'sum'  # Sum the spend for each group
    }).reset_index()
    grouped_df = grouped_df.rename(columns={
        'cust_id': 'distinct_cust_count',
        'spend_rand': 'total_spend'
    })
    grouped_df['date'] = pd.to_datetime(grouped_df['date'])
    grouped_df.sort_values(by='date', inplace=True)
    return grouped_df

def calculate_alive_percentage(df):
    """
    Calculates '%Alive' column for the DataFrame.
    """
    base_value = df.loc[df['cohort'] == 0, 'distinct_cust_count'].iloc[0]
    df['%Alive'] = df['distinct_cust_count'] / base_value
    return df


def e_alive(cohort, gamma, delta):
    """
    E(% Alive) function.
    """
    # Convert cohort to numpy array if it's not already
    cohort_array = np.array(cohort) if not isinstance(cohort, np.ndarray) else cohort

    # Debugging print statements
    #print("Shapes and types:")
    #print("Cohort:", cohort_array.shape, type(cohort_array))
    #print("Gamma:", gamma, type(gamma))
    #print("Delta:", delta, type(delta))

    # Perform element-wise operations
    gamma_delta_sum = gammaln(gamma + delta)
    return np.exp(gammaln(delta + cohort_array) + gamma_delta_sum - gammaln(delta) - gammaln(gamma + delta + cohort_array))


def calculate_values(df, gamma, delta):
    """
    Calculates E(% Alive), P(ChurnTime = t), and E(# of Cust) for the DataFrame.
    """
    df['E(% Alive)'] = e_alive(df['cohort'], gamma, delta)
    df['P(ChurnTime = t)'] = df['E(% Alive)'].diff().fillna(df['E(% Alive)'].iloc[0])
    df['E(# of Cust)'] = df['distinct_cust_count'].iloc[0] * df['E(% Alive)']
    return df

def sse(params, df):
    """
    SSE function for optimization.
    """
    gamma, delta = params
    df_temp = calculate_values(df.copy(), gamma, delta)
    return np.sum((df_temp['E(# of Cust)'] - df_temp['distinct_cust_count']) ** 2)

def optimize_gamma_delta(df, initial_guess):
    """
    Optimizes gamma and delta values using the SSE function.
    """
    result = minimize(sse, initial_guess, args=(df,), bounds=[(0.00001, None), (0.00001, None)])
    return result.x

def calculate_e_lifetime_years(df, column='t * P(ChurnTime = t)'):
    """
    Calculates the expected lifetime (E(Lifetime)) in years.
    """
    total = df[column].sum()
    e_lifetime_years = total / 12
    return  e_lifetime_years

def calculate_e_lifetime_years_3mo(df, column='t * P(churnTime = t | Alive @ 3)'):
    """
    Calculates the expected lifetime (E(Lifetime)) in years.
    """
    total = df[column].sum()
    e_lifetime_years_3mo = total / 12
    return  e_lifetime_years_3mo

def calculate_new_column(row):
    """
    Calculates the new column 't * P(ChurnTime = t)' for a given row.
    """
    # Use 'cohort' directly since it's an integer
    return row['cohort'] * row['P(ChurnTime = t)']

def append_new_row(df):
    """
    Appends a new row to the dataframe with specified calculations.

    :param df: The dataframe to which the row will be appended.
    :param initial_cust_count: The initial customer count, used for 'E(# of Cust)' calculation.
    :return: The dataframe with the new row appended.
    """
    new_row = {
        'cohort_month': '> 120',  # as specified
        'cohort': 121,  # as specified
        't * P(ChurnTime = t)': df['t * P(ChurnTime = t)'].sum(),
        # Sum all values for 'P(ChurnTime = t)' to get the new value
        'P(ChurnTime = t)': 2 - df['P(ChurnTime = t)'].sum(),
        # Calculate the new 'E(% Alive)'
        'E(% Alive)': 2 - df['P(ChurnTime = t)'].sum() - df['E(% Alive)'].iloc[-1],
        'E(# of Cust)': df['distinct_cust_count'].iloc[0] * (2 - df['P(ChurnTime = t)'].sum() - df['E(% Alive)'].iloc[-1])
    }
    new_row_df = pd.DataFrame([new_row])
    df = pd.concat([df, new_row_df], ignore_index=True)
    return df


def calculate_conditional_probability(df):
    """
    Calculates the conditional probability 'P(churnTime = t | Alive @ 3)'.
    """
    # Get the 'E(% Alive)' value for cohort 3
    alive_at_3 = df.loc[df['cohort'] == 3, 'E(% Alive)'].values[0]

    # Define a function to apply to each row
    def calculate_probability(row):
        if row['cohort'] <= 3:
            return 0
        else:
            return   row['P(ChurnTime = t)'] / alive_at_3

    # Apply the function to each row to create the new column
    df['P(churnTime = t | Alive @ 3)'] = df.apply(calculate_probability, axis=1)

    return df

def calculate_filtered_sse(df, actual_column, predicted_column):
    """
    Filters the DataFrame for non-NaN values in 'distinct_cust_count' and calculates SSE.
    """
    # Filter DataFrame based on non-NaN values in 'distinct_cust_count'
    filtered_df = df[df[actual_column].notna()]

    # Extract actual and predicted values
    actual = filtered_df[actual_column].values
    predicted = filtered_df[predicted_column].values

    # Calculate SSE
    sse = np.sum((actual - predicted) ** 2)

    return sse

def calculate_t_times_conditional_probability(df, conditional_prob_column='P(churnTime = t | Alive @ 3)', cohort_column='cohort'):
    """
    Calculates the new column 't * P(churnTime = t | Alive @ 3)'.
    
    :param df: The dataframe containing the data.
    :param conditional_prob_column: The name of the column containing 'P(churnTime = t | Alive @ 3)' values.
    :param cohort_column: The name of the column containing cohort values.
    :return: The dataframe with the new column added.
    """
    # Calculate the new column by multiplying 'P(churnTime = t | Alive @ 3)' by 'cohort'
    new_column_name = f"t * {conditional_prob_column}"
    df[new_column_name] = df[cohort_column] * df[conditional_prob_column]
    
    return df

def calculate_financial_metrics(data, margin=0.20, WACC_monthly=0.0095):
    """
    Calculate various financial metrics based on the provided dataset.

    Parameters:
    data (DataFrame): Pandas DataFrame containing the data.
    margin (float): Margin for calculating profit (default: 20%).
    WACC_monthly (float): Monthly Weighted Average Cost of Capital (default: 0.95%).

    Returns:
    DataFrame: Updated DataFrame with new financial metrics.
    """
    data['ARPU'] = data['total_spend'] / data['distinct_cust_count']

    # Define the model
    def model(params, X):
        return params[0] + params[1] * X

    # Define the SSE function
    def sse(params, X, Y):
        return np.sum((Y - model(params, X)) ** 2)

    # Extract variables
    X = data['cohort']
    Y = data['ARPU']

    # Initial guess for parameters
    initial_guess = [0, 0]

    # Optimize parameters
    result = minimize(sse, initial_guess, args=(X, Y))
    if not result.success:
        raise ValueError(result.message)

    fitted_params = result.x
    b0 = {fitted_params[0]}
    b1 = {fitted_params[1]}


    # Calculate additional metrics
    
    data['Calculated_ARPU'] = model(fitted_params, X)
    data['E(Total Rev)'] = data['Calculated_ARPU'] * data['distinct_cust_count']
    data['E(Rev. per Acq. Cust)'] = data['Calculated_ARPU'] * data['E(% Alive)']
    data['E(Profit per Acq. Cust)'] = data['E(Rev. per Acq. Cust)'] * margin
    data['PV( E(Profit per Acq. Cust) )'] = data['E(Profit per Acq. Cust)'] / ((1 + WACC_monthly) ** data['cohort'])

    return data,b0, b1

In [50]:
# Declaring the file
# Main Script
file_path = "rich_sample_data.csv"


In [67]:
# Running the main calculation to have the final table df_final
try:
    df_filtered = load_and_preprocess_data(file_path)
except FileNotFoundError:
    print("File not found. Please check the file path and try again.")

grouped_df = calculate_distinct_customer_count(df_filtered)
grouped_df['cohort'] = range(len(grouped_df))
grouped_df = calculate_alive_percentage(grouped_df)

# Optimization
initial_guess = [1, 1]
optimized_gamma, optimized_delta = optimize_gamma_delta(grouped_df, initial_guess)

grouped_df = calculate_values(grouped_df, optimized_gamma, optimized_delta)


# Extend the cohort values up to 24 using vectorized operations
max_cohort = grouped_df['cohort'].max()
extended_cohorts = pd.DataFrame({'cohort': range(max_cohort + 1, 121), '%Alive': np.nan})
grouped_df = pd.concat([grouped_df, extended_cohorts], ignore_index=True)

# Calculate E(% Alive) for the entire range of cohorts
grouped_df['E(% Alive)'] = e_alive(grouped_df['cohort'], optimized_gamma, optimized_delta)

# Calculate P(ChurnTime = t) and E(# of Cust)
initial_cust_count = grouped_df['distinct_cust_count'].iloc[0]
grouped_df['P(ChurnTime = t)'] = grouped_df['E(% Alive)'].diff().fillna(grouped_df['E(% Alive)'].iloc[0])
grouped_df['E(# of Cust)'] = initial_cust_count * grouped_df['E(% Alive)']

# Adding cohort month
from pandas.tseries.offsets import MonthBegin
start_month = grouped_df['date'].min()
grouped_df['cohort_month'] = grouped_df.apply(lambda row: start_month + MonthBegin(n=int(row['cohort'])), axis=1)

updated_sample_data, b0,b1 = calculate_financial_metrics(grouped_df)
print(b0)

# Final DataFrame
df_final = grouped_df[['cohort_month', 'cohort', 'total_spend', 'ARPU','distinct_cust_count', '%Alive', 'E(% Alive)', 
                       'P(ChurnTime = t)', 'E(# of Cust)','Calculated_ARPU', 'E(Total Rev)', 'E(Rev. per Acq. Cust)',
       'E(Profit per Acq. Cust)', 'PV( E(Profit per Acq. Cust) )']]
df_final['P(ChurnTime = t)'] = np.abs(df_final['P(ChurnTime = t)'])
df_final['t * P(ChurnTime = t)'] = df_final.apply(calculate_new_column, axis=1)
df_final = append_new_row(df_final)
# Apply the function to df_final
df_final = calculate_conditional_probability(df_final)
df_final = calculate_t_times_conditional_probability(df_final)
df_final = df_final[['cohort_month', 'cohort','distinct_cust_count','%Alive', 
                     'E(% Alive)', 'P(ChurnTime = t)', 'E(# of Cust)','t * P(ChurnTime = t)',	'P(churnTime = t | Alive @ 3)','t * P(churnTime = t | Alive @ 3)',
                     'total_spend', 'ARPU', 'Calculated_ARPU', 'E(Total Rev)', 'E(Rev. per Acq. Cust)','E(Profit per Acq. Cust)', 'PV( E(Profit per Acq. Cust) )']]
df_final




C:\Users\gusdorne\AppData\Local\Temp\ipykernel_19024\2577021119.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.sort_values(by='date', inplace=True)
C:\Users\gusdorne\AppData\Local\Temp\ipykernel_19024\1264120466.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['P(ChurnTime = t)'] = np.abs(df_final['P(ChurnTime = t)'])


,cohort_month,cohort,distinct_cust_count,%Alive,E(% Alive),P(ChurnTime = t),E(# of Cust),t * P(ChurnTime = t),P(churnTime = t | Alive @ 3),t * P(churnTime = t | Alive @ 3),total_spend,ARPU,Calculated_ARPU,E(Total Rev),E(Rev. per Acq. Cust),E(Profit per Acq. Cust),PV( E(Profit per Acq. Cust) )
0,2018-01-01 00:00:00,0,987.0,1.000000,1.000000,1.000000,987.000000,0.000000,0.000000,0.000000,29543.292828,29.932414,29.804794,29417.331321,29.804794,5.960959,5.960959
1,2018-02-01 00:00:00,1,869.0,0.880446,0.849096,0.150904,838.058159,0.150904,0.000000,0.000000,25700.816139,29.575162,29.835556,25927.097847,25.333263,5.066653,5.018972
2,2018-03-01 00:00:00,2,743.0,0.752786,0.746902,0.102194,737.192741,0.204388,0.000000,0.000000,21986.084005,29.590961,29.866318,22190.674000,22.307226,4.461445,4.377871
3,2018-04-01 00:00:00,3,653.0,0.661601,0.672142,0.074760,663.404149,0.224281,0.000000,0.000000,19789.710278,30.305835,29.897080,19522.792997,20.095083,4.019017,3.906617
4,2018-05-01 00:00:00,4,593.0,0.600811,0.614559,0.057583,606.569524,0.230333,0.085671,0.342685,17736.798698,29.910284,29.927842,17747.210083,18.392418,3.678484,3.541959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,2027-10-01 00:00:00,117,NaN,NaN,0.114781,0.000572,113.289314,0.066903,0.000851,0.099537,NaN,NaN,33.403947,NaN,3.834154,0.766831,0.253665
118,2027-11-01 00:00:00,118,NaN,NaN,0.114217,0.000564,112.732371,0.066585,0.000840,0.099064,NaN,NaN,33.434709,NaN,3.818819,0.763764,0.250272
119,2027-12-01 00:00:00,119,NaN,NaN,0.113660,0.000557,112.182710,0.066271,0.000829,0.098597,NaN,NaN,33.465471,NaN,3.803695,0.760739,0.246935
120,2028-01-01 00:00:00,120,NaN,NaN,0.113111,0.000550,111.640178,0.065961,0.000818,0.098136,NaN,NaN,33.496233,NaN,3.788780,0.757756,0.243652


In [58]:
# Getting some important values: Lifetime, Lifetime_3mo, future_Lifetime
e_lifetime_years = f"{calculate_e_lifetime_years(df_final):.4f}"
e_lifetime_years_3mo_v = f"{calculate_e_lifetime_years_3mo(df_final):.4f}"

cohort_nr =  3  ### USER SET

e_future_lifetime_years_3mo = calculate_e_lifetime_years_3mo(df_final) - (cohort_nr/12)
e_future_lifetime_years_3mo =  f"{e_future_lifetime_years_3mo:.4f}"



sse_value_1 = calculate_filtered_sse(df_final, actual_column='distinct_cust_count', predicted_column='E(# of Cust)')
sse_value_1 # Customer

sse_value_2 = calculate_filtered_sse(df_final, actual_column='total_spend', predicted_column='E(Total Rev)')
sse_value_2 # Revenue

e_lifetime_years, e_lifetime_years_3mo_v, e_future_lifetime_years_3mo, sse_value_1, sse_value_2

('2.2455', '3.2954', '3.0454', 1527.268367392978, 146787.79372692984)

In [54]:
sse_value_1 = calculate_filtered_sse(df_final, actual_column='distinct_cust_count', predicted_column='E(# of Cust)')
sse_value_1 # Customer

1527.268367392978

In [65]:
b0, b1

NameError: name 'b0' is not defined

In [61]:
# Define the model
def model(params, X):
    b_00, b_01 = params
    return b_00 + b_01 * X

# Define the SSE (Sum of Squared Errors) function to minimize
def sse(params, X, Y):
    return np.sum((Y - model(params, X)) ** 2)

X = grouped_df['cohort']
Y = grouped_df['ARPU']

    # Initial guess for parameters
initial_guess = [0, 0]
result = minimize(sse, initial_guess, args=(X, Y))
    # Optimize parameters
if result.success:
    fitted_params = result.x
    print(f"Optimized parameters: b_00 = {fitted_params[0]}, b_01 = {fitted_params[1]}")

    # Create new column for calculated ARPU
    grouped_df['Calculated_ARPU'] = model(fitted_params, X)
else:
    raise ValueError(result.message)

fitted_params = result.x

Optimized parameters: b_00 = 30.023190855537333, b_01 = 0.0012976780864855872
